In [2]:
%load_ext autoreload
%autoreload 2

In [48]:
import numpy as np
import json
import pickle
import argparse
from nltk.tokenize import word_tokenize
from process_data import save_pickle, load_pickle, load_task, load_glove_weights, to_var, make_word_vector
from word_embedding import WordEmbedding
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import sys

In [29]:
sys.argv = ['a.py']

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=8, help='input batch size')
parser.add_argument('--embd_size', type=int, default=100, help='word embedding size')
args = parser.parse_args()

In [54]:
# train_data = load_task('./dataset/train-v1.1.json')
train_data = load_pickle('./pickles/train_data.pickle')
# dev_data = load_task('./dataset/dev-v1.1.json')
dev_data = load_pickle('./pickles/dev_data.pickle')
data = train_data+dev_data
vocab, vocab_a = set(), set()
for ctx, q_id, query,answer in data:
    vocab |= set(ctx + query)# + answer)
    vocab_a |= set(answer)
    
vocab = list(sorted(vocab))
vocab_a = list(sorted(vocab_a))
w2i = dict((w, i) for i, w in enumerate(vocab, 0))
i2w = dict((i, w) for i, w in enumerate(vocab, 0))
a2i = dict((w, i) for i, w in enumerate(vocab_a, 0))
i2a = dict((i, w) for i, w in enumerate(vocab_a, 0))
args.vocab_size = len(vocab)
args.ans_size = len(vocab_a)
print('vocab size', len(vocab))
print('ans size', len(vocab_a))

ctx_maxlen = max([len(c) for c, _, _, _ in data])
query_maxlen = max([len(q) for _, _, q, _ in data])
print('context maxlen:', ctx_maxlen)
print('query maxlen:', query_maxlen)

glove_embd = load_pickle('./pickles/glove_embd.pickle')
# glove_embd = torch.from_numpy(load_glove_weights('./dataset', args.embd_size, len(vocab), w2i)).type(torch.FloatTensor)
# save_pickle(glove_embd, './pickles/glove_embd.pickle')
args.pre_embd = glove_embd

load ./pickles/train_data.pickle
load ./pickles/dev_data.pickle
vocab size 123295
ans size 78025
context maxlen: 766
query maxlen: 60
load ./pickles/glove_embd.pickle


In [64]:
class JNet(nn.Module):
    def __init__(self, args):
        super(JNet, self).__init__()
        self.embd_size = args.embd_size
        
        self.embd = WordEmbedding(args)
        self.ctx_birnn = nn.GRU(self.embd_size, self.embd_size, bidirectional=True, dropout=0.2)
        self.query_birnn = nn.GRU(self.embd_size, self.embd_size, bidirectional=True, dropout=0.2)
        
        
    def forward(self, context, query):
        print('context', context.size())
        print('query', query.size())
        context_data = self.embd(context)
        print('context_embd', context_data.size())
        context_data, _h = self.ctx_birnn(context_data)
        print('context_gru', context_data.size())
        
        query_data = self.embd(query)
        print('query embd', query_data.size())
        query_data, _h = self.ctx_birnn(query_data)
        print('query gru', query_data.size())
        

        
def train(model, optimizer=None, n_epoch=1, batch_size=16):
    for epoch in range(n_epoch):
        print('---Epoch', epoch)
        for i in range(0, len(data)-batch_size, batch_size): # TODO shuffle, last elms
            batch_data = data[i:i+batch_size]
            c = [d[0] for d in batch_data]
            q = [d[2] for d in batch_data]
            context_var = make_word_vector(c, w2i, ctx_maxlen)
            query_var = make_word_vector(q, w2i, query_maxlen)
            ans_var = to_var(torch.LongTensor([a2i[d[3][0]] for d in batch_data]))#.squeeze()) #
            model(context_var, query_var)
            break
        break
            
#             model.zero_grad()
#             loss.backward()
#             optimizer.step()
    
model = JNet(args)
# if torch.cuda.is_available():
#     model.cuda()
# print(model)
# for p in model.parameters():
#     print(p)
# optimizer = torch.optim.Adadelta(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=0.5, weight_decay=0.999)
train(model)#, optimizer)
print('fin')

---Epoch 0
context torch.Size([16, 766])
query torch.Size([16, 60])
context_embd torch.Size([16, 766, 100])
context_gru torch.Size([16, 766, 200])
query embd torch.Size([16, 60, 100])
query gru torch.Size([16, 60, 200])
fin
